<a href="https://colab.research.google.com/github/mosovam/nn-medical-view/blob/main/Model_to_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/8c7f0be1e1c3803fcb4c41bcd9f4226b/super_resolution_with_onnxruntime.ipynb

https://pytorch.org/docs/master/onnx.html#example-alexnet-from-pytorch-to-onnx

# Imports and installations

In [ ]:
%pip install segmentation-models-pytorch

     |████████████████████████████████| 88 kB 3.4 MB/s 
     |████████████████████████████████| 376 kB 56.8 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=daf34d6c9e12efc201779b3bbc615ef47d24949dde2200222964f682162e097c
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=8fc8cf662b090c6dfb46501a3c0e99585765791f8402873db82695eb65316dc5
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
# Some standard imports

import io
import numpy as np
import segmentation_models_pytorch as smp

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [ ]:
# google drive connection

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Convert model for tumor segmentation

In [ ]:
# prepare the model with predefiined parameters used when training the model

FOLDER_PATH = '/content/drive/MyDrive/OSU_IT/BC_MMO'
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
CLASSES = ['tumor', 'unlabelled']

my_model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION
)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


  0%|          | 0.00/105M [00:00<?, ?B/s]

In [ ]:
# import the specific model and eval()

path = FOLDER_PATH + '/tumor_model/my_segmentation_model_saved_as_epochs8'

my_model.load_state_dict(torch.load(path))
my_model.eval()

Unet(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
if torch.cuda.is_available():
    print('Model on cuda!')
    my_model.cuda()

Model on cuda!


In [ ]:
# Export the model
## use the specific images shapes - 1, 3, 512, 512

dummy_input = torch.randn(1, 3, 512, 512, device="cuda")
torch.onnx.export(my_model,                                         # model being run
                  dummy_input,                                      # model input (or a tuple for multiple inputs)
                  FOLDER_PATH + "/brain_tumor_segmentation.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,                               # store the trained parameter weights inside the model file,
                  opset_version=12,                                 # the ONNX version to export the model to
                  input_names = ['input'],                          # the model's input names
                  output_names = ['output'],                        # the model's output names
                  operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK
)

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:269: UserWarning: `add_node_names' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `add_node_names` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:269: UserWarning: `do_constant_folding' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `do_constant_folding` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
